In [ ]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.transforms.functional as f
torch.set_float32_matmul_precision('medium')

In [ ]:
class SharpenTransform:
    def __init__(self, sharpness_factor=2.0):
        self.sharpness_factor = sharpness_factor

    def __call__(self, img):
        return f.adjust_sharpness(img, self.sharpness_factor)

In [ ]:
class FERPlusDataset(Dataset):
    def __init__(self, img_folder, csv_file, transform=None, skip_nf=True):
        """
        Args:
            img_folder (str): Directory containing the image files.
            csv_file (str): Path to CSV annotation file containing image names, labels, and emotion distributions.
            transform (callable, optional): Optional torchvision transforms to apply on image.
            skip_nf (bool): Whether to exclude "NF" (Not Face) images from dataset.
            
        Notes:
            - The CSV must include columns for image name, Usage split (Training, PublicTest, PrivateTest),
              emotion count columns matching 'neutral', 'happiness', 'surprise', 'sadness', 'anger', 
              'disgust', 'fear', 'contempt', 'unknown', and the 'NF' label.
            - Images are loaded as grayscale, resized to 224x224, and sharpened with factor 5 by default.
            - Label distribution is computed by normalizing the raw emotion counts per sample.
        """
        self.img_folder = img_folder
        self.transform = transform
        
        # Load CSV with label distributions and metadata
        self.data = pd.read_csv(csv_file)
        
        # Map folder name to 'Usage' column value in CSV to filter dataset split accordingly
        usage_map = {
            "FER2013Train": "Training",
            "FER2013Valid": "PublicTest",
            "FER2013Test": "PrivateTest"
        }
        # Identify which subset (split) to load based on folder name
        usage_required = usage_map.get(os.path.basename(img_folder), None)
        if usage_required is None:
            raise ValueError(f"Unknown dataset folder: {img_folder}")
        
        # Filter the dataset to only include images from the selected split
        self.data = self.data[self.data['Usage'] == usage_required].reset_index(drop=True)
        
        # Optionally exclude images marked as Not Face (NF = 1)
        if skip_nf:
            self.data = self.data[self.data['NF'] == 0].reset_index(drop=True)
        
        # Define emotion categories (order must match columns in CSV exactly)
        self.emotions = ['neutral','happiness','surprise','sadness','anger','disgust','fear','contempt','unknown']
        
        # If no transform provided, define default preprocessing:
        # Convert to 224x224 grayscale, apply sharpening with factor 5
        if self.transform is None:
            self.transform = transforms.Compose([
                transforms.Resize((224, 224), interpolation=transforms.InterpolationMode.LANCZOS),
                SharpenTransform(sharpness_factor=5),
                transforms.ToTensor(),   # convert PIL Image to tensor after preprocessing
                transforms.Normalize(mean=[0.5076], std=[0.2119])
            ])


    def __len__(self):
        # Return total number of valid samples
        return len(self.data)

    def __getitem__(self, idx):
        # Get image filename at index
        img_name = self.data.loc[idx, 'Image name']

        # Construct full image path
        img_path = os.path.join(self.img_folder, img_name)
        
        # Load image as grayscale with PIL
        image = Image.open(img_path).convert('L')
        
        # Apply preprocessing transforms (resize, sharpen, to tensor)
        image = self.transform(image)

        # Extract raw emotion counts for label distribution
        counts = self.data.loc[idx, self.emotions].values.astype(float)
        
        # Normalize counts to create label distribution (probability vector)
        label_distribution = counts / counts.sum()
        
        # Convert label distribution to PyTorch tensor of floats
        label_distribution = torch.tensor(label_distribution, dtype=torch.float32)
        
        # Return preprocessed image tensor and LDL label distribution
        return image, label_distribution

In [ ]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

# Paths remain the same
train_imgs_path = "../FERPlusData/FER2013Train"
val_imgs_path = "../FERPlusData/FER2013Valid"
test_imgs_path = "../FERPlusData/FER2013Test"
data_csv_path = "../fer2013new.csv"

# Define the sharpening transform once to reuse
sharpen = SharpenTransform(sharpness_factor=5.0)

# Training dataset and loader with full augmentations
train_dataset = FERPlusDataset(
    img_folder=train_imgs_path,
    csv_file=data_csv_path,
    skip_nf=True,
    transform=transforms.Compose([
        transforms.Resize((224, 224), interpolation=transforms.InterpolationMode.LANCZOS),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=10),
        transforms.RandomAffine(degrees=5, translate=(0.05, 0.05), scale=(0.95, 1.05), shear=5),
        sharpen,
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5076], std=[0.2119])
    ])
)

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
)

# Validation dataset and loader with only deterministic preprocessing
val_dataset = FERPlusDataset(
    img_folder=val_imgs_path,
    csv_file=data_csv_path,
    skip_nf=True,
    transform=transforms.Compose([
        transforms.Resize((224, 224), interpolation=transforms.InterpolationMode.LANCZOS),
        sharpen,
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5076], std=[0.2119])
    ])
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
)

In [ ]:
# CBAM Module
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=8):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc1 = nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        return self.sigmoid(avg_out + max_out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        padding = (kernel_size - 1) // 2
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x_cat = torch.cat([avg_out, max_out], dim=1)
        return self.sigmoid(self.conv1(x_cat))

class CBAM(nn.Module):
    def __init__(self, in_planes, ratio=8, spatial_kernel_size=7):
        super(CBAM, self).__init__()
        self.ca = ChannelAttention(in_planes, ratio)
        self.sa = SpatialAttention(spatial_kernel_size)

    def forward(self, x):
        out = x * self.ca(x)
        out = out * self.sa(out)
        return out

In [ ]:
# ECA Module
class ECA(nn.Module):
    def __init__(self, in_channels, gamma=2, b=1):
        super(ECA, self).__init__()
        t = int(abs((torch.log2(torch.tensor(in_channels, dtype=torch.float32)) + b) / gamma))
        k_size = t if t % 2 else t + 1
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv = nn.Conv1d(1, 1, kernel_size=k_size, padding=(k_size - 1) // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        y = self.avg_pool(x)
        y = y.squeeze(-1).transpose(-1, -2)
        y = self.conv(y)
        y = self.sigmoid(y).transpose(-1, -2).unsqueeze(-1)
        return x * y.expand_as(x)

In [ ]:
# Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1, dropout=False, p=0.2):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, padding=padding, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.dropout = nn.Dropout2d(p=p) if dropout else nn.Identity()

        self.skip = nn.Conv2d(in_channels, out_channels, 1, bias=False) if in_channels != out_channels else nn.Identity()

    def forward(self, x):
        identity = self.skip(x)
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity
        out = self.relu(out)
        return out

In [ ]:
class TwoBranchFERTransformer(nn.Module):
    def __init__(self, num_classes=9):
        super().__init__()
        
        # Branch 1 (fine details - 3x3 kernels)
        self.b1_block1 = ResidualBlock(1, 32, kernel_size=3, padding=1)
        self.b1_pool1 = nn.MaxPool2d(2)
        self.b1_block2 = ResidualBlock(32, 64, kernel_size=3, padding=1)
        self.b1_pool2 = nn.MaxPool2d(2)
        self.b1_block3 = ResidualBlock(64, 128, kernel_size=3, padding=1, dropout=True, p=0.1)
        self.b1_pool3 = nn.MaxPool2d(2)
        self.b1_block4 = ResidualBlock(128, 128, kernel_size=3, padding=1, dropout=True, p=0.1)
        self.cbam_b1 = CBAM(128, ratio=8)  # Strong attention on branch 1 output (reduction=4)

        # Branch 2 (broad details - 5x5 kernels)
        self.b2_block1 = ResidualBlock(1, 32, kernel_size=5, padding=2)
        self.b2_pool1 = nn.MaxPool2d(2)
        self.b2_block2 = ResidualBlock(32, 64, kernel_size=5, padding=2)
        self.b2_pool2 = nn.MaxPool2d(2)
        self.b2_block3 = ResidualBlock(64, 128, kernel_size=5, padding=2, dropout=True, p=0.1)
        self.b2_pool3 = nn.MaxPool2d(2)
        self.b2_block4 = ResidualBlock(128, 128, kernel_size=5, padding=2, dropout=True, p=0.1)
        self.cbam_b2 = CBAM(128, ratio=8)  # Strong attention on branch 2 output (reduction=4)

        # Learnable fusion weights for branches after CBAM
        self.w1 = nn.Parameter(torch.tensor(0.5))
        self.w2 = nn.Parameter(torch.tensor(0.5))

        # Final CBAM on fused features (milder attention, reduction=16)
        self.cbam_fused = CBAM(128, ratio=16)

        # Transformer encoder: 2 layers, 2 heads, embedding 128
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=128, nhead=2, dim_feedforward=256, dropout=0.2, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=1)

        # Final classification fully connected layer for 9 classes
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        # Branch 1 forward with pooling
        b1 = self.b1_block1(x)
        b1 = self.b1_pool1(b1)
        b1 = self.b1_block2(b1)
        b1 = self.b1_pool2(b1)
        b1 = self.b1_block3(b1)
        b1 = self.b1_pool3(b1)
        b1 = self.b1_block4(b1)
        b1 = self.cbam_b1(b1)

        # Branch 2 forward with pooling
        b2 = self.b2_block1(x)
        b2 = self.b2_pool1(b2)
        b2 = self.b2_block2(b2)
        b2 = self.b2_pool2(b2)
        b2 = self.b2_block3(b2)
        b2 = self.b2_pool3(b2)
        b2 = self.b2_block4(b2)
        b2 = self.cbam_b2(b2)

        # Weighted fusion of the two branch outputs
        fused = self.w1 * b1 + self.w2 * b2

        # Final CBAM on fused features (milder attention)
        fused = self.cbam_fused(fused)
        fused = F.dropout(fused, p=0.2, training=self.training)

        # Prepare transformer input: [batch, seq_len, embed_dim]
        B, C, H, W = fused.shape
        out = fused.permute(0, 2, 3, 1).reshape(B, H * W, C)

        # Transformer encoder
        out = self.transformer(out)

        # Global average pooling over sequence dimension
        out = out.mean(dim=1)

        # Final classification layer
        logits = self.fc(out)

        # Softmax for label distribution output
        return torch.softmax(logits, dim=1)

In [ ]:
# 1. Device
device = torch.device(
    "cuda" if torch.cuda.is_available() 
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using device: {device}")

# 2. Max epochs
max_epochs = 50

# 3. Model & Optimizer
model = TwoBranchFERTransformer(num_classes=9).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

# 4. LR Scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, 
    T_max=max_epochs,
    eta_min=1e-6
)

In [ ]:
import numpy as np
class EarlyStopper:
    """
    Early stops training if validation loss doesn't improve after a given patience.
    """
    def __init__(self, patience=7, min_delta=0.0, verbose=False, save_path=None):
        """
        Args:
            patience (int): How long to wait after last improvement before stopping.
            min_delta (float): Minimum change to qualify as improvement.
            verbose (bool): Print messages when validation improves.
            save_path (str or None): Path to save the best model checkpoint.
        """
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        self.counter = 0
        self.best_loss = np.Inf
        self.early_stop = False
        self.save_path = save_path

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            if self.save_path is not None:
                torch.save(model.state_dict(), self.save_path)
                if self.verbose:
                    print(f"Validation loss decreased. Saving model to {self.save_path}")
        else:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True


In [ ]:
def loss_fn(logits, label_distributions):
    log_probs = F.log_softmax(logits, dim=1)
    loss = F.kl_div(log_probs, label_distributions, reduction='batchmean')
    return loss

In [ ]:
from torchmetrics.classification import MulticlassAccuracy, MulticlassF1Score
from torchmetrics import MeanSquaredError
from tqdm import tqdm

def train_val_loop(
    model,
    train_loader,
    val_loader,
    optimizer,
    scheduler,
    loss_fn,
    early_stopper,
    num_epochs,
    device
):
    # Initialize torchmetrics for accuracy, f1 and mse on the device
    train_acc = MulticlassAccuracy(num_classes=9, average='micro').to(device)
    val_acc = MulticlassAccuracy(num_classes=9, average='micro').to(device)

    train_f1 = MulticlassF1Score(num_classes=9, average='weighted').to(device)
    val_f1 = MulticlassF1Score(num_classes=9, average='weighted').to(device)

    train_mse = MeanSquaredError().to(device)
    val_mse = MeanSquaredError().to(device)

    scaler = torch.amp.GradScaler()
    use_amp = (device.type == 'cuda')

    for epoch in range(1, num_epochs + 1):
        # --- Training Phase ---
        model.train()
        total_train_loss = 0
        train_acc.reset()
        train_f1.reset()
        train_mse.reset()
        train_samples = 0

        train_bar = tqdm(train_loader, desc=f"Epoch {epoch} Training", leave=False)
        for images, labels in train_bar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            with torch.amp.autocast(enabled=use_amp, device_type=device.type):
                logits = model(images)
                loss = loss_fn(logits, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_train_loss += loss.item() * images.size(0)
            train_samples += images.size(0)

            probs = F.softmax(logits, dim=1)
            preds = torch.argmax(probs, dim=1)
            targets = torch.argmax(labels, dim=1)

            train_acc.update(preds, targets)
            train_f1.update(preds, targets)
            train_mse.update(probs, labels)

            train_bar.set_postfix(loss=loss.item())

        avg_train_loss = total_train_loss / train_samples
        avg_train_acc = train_acc.compute().item()
        avg_train_f1 = train_f1.compute().item()
        avg_train_mse = train_mse.compute().item()

        # --- Validation Phase ---
        model.eval()
        total_val_loss = 0
        val_acc.reset()
        val_f1.reset()
        val_mse.reset()
        val_samples = 0

        with torch.no_grad():
            val_bar = tqdm(val_loader, desc=f"Epoch {epoch} Validation", leave=False)
            for images, labels in val_bar:
                images, labels = images.to(device), labels.to(device)
                with torch.amp.autocast(enabled=use_amp, device_type=device.type):
                    logits = model(images)
                    loss = loss_fn(logits, labels)
                total_val_loss += loss.item() * images.size(0)
                val_samples += images.size(0)

                probs = F.softmax(logits, dim=1)
                preds = torch.argmax(probs, dim=1)
                targets = torch.argmax(labels, dim=1)

                val_acc.update(preds, targets)
                val_f1.update(preds, targets)
                val_mse.update(probs, labels)

                val_bar.set_postfix(loss=loss.item())

        avg_val_loss = total_val_loss / val_samples
        avg_val_acc = val_acc.compute().item()
        avg_val_f1 = val_f1.compute().item()
        avg_val_mse = val_mse.compute().item()

        # Step learning rate scheduler once per epoch after validation
        scheduler.step()

        # Print training and validation results for the epoch
        print(f"[Epoch {epoch}/{num_epochs}] "
              f"Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f} | "
              f"Train Acc: {avg_train_acc:.4f}, Val Acc: {avg_val_acc:.4f} | "
              f"Train F1: {avg_train_f1:.4f}, Val F1: {avg_val_f1:.4f} | "
              f"Train MSE: {avg_train_mse:.4f}, Val MSE: {avg_val_mse:.4f}")

        # Early stopping check and save best model
        early_stopper(avg_val_loss, model)
        if early_stopper.early_stop:
            print("Early stopping triggered. Stopping training.")
            break

    # Load the best weights saved by early stopper (if any)
    if early_stopper.save_path is not None:
        print(f"Loading best model from {early_stopper.save_path}")
        model.load_state_dict(torch.load(early_stopper.save_path, map_location=device))

In [ ]:
early_stopper = EarlyStopper(
    patience=10,
    min_delta=1e-4,
    verbose=True,
    save_path="../models/early_best_models/new_arch_best_model.pth"
)

train_val_loop(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    scheduler=scheduler,
    loss_fn=loss_fn,
    early_stopper=early_stopper,
    num_epochs=max_epochs,
    device=device
)


In [ ]:
model.eval()

# Create a dummy input tensor with the same shape as your model input
# For example, if your images are grayscale 224x224 and batch size 1:
dummy_input = torch.randn(1, 1, 224, 224, device=device)

# Define the path where to save the ONNX model
onnx_path = "../models/new_arch.onnx"

# Export the model
torch.onnx.export(
    model,                      # model being run
    dummy_input,                # model input (or a tuple for multiple inputs)
    onnx_path,                  # where to save the model
    export_params=True,         # store the trained parameter weights inside the model file
    opset_version=12,           # ONNX version to export the model to
    do_constant_folding=True,   # whether to execute constant folding for optimization
    input_names=['input'],      # the model's input names
    output_names=['output'],    # the model's output names
    dynamic_axes={              # variable length axes
        'input': {0: 'batch_size'},   # allow variable batch size for input
        'output': {0: 'batch_size'}   # allow variable batch size for output
    }
)

print(f"Model has been successfully exported to {onnx_path}")